<a href="https://colab.research.google.com/github/codereyinish/RAG_COMMENTER/blob/main/CommenterRAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade pip

  Using cached pip-24.2-py3-none-any.whl.metadata (3.6 kB)
Using cached pip-24.2-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [2]:
!pip install llama-index
!pip install transfomers
!pip install llama-index-embeddings-huggingface
!pip install peft
!pip install auto-gptq
!pip install optimum
!pip install bitsandbytes



INFO: pip is looking at multiple versions of llama-index-indices-managed-llama-cloud to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of llama-index-readers-llama-parse to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 45.7 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
ERROR: Could not find a version that satisfies the requirement transfomers (from versions: none)
ERROR: No matching distribution found for transfomers
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvi

In [3]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings, SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /usr/local/lib/python3.10/dist-
[nltk_data]     packages/llama_index/core/_static/nltk_cache...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [4]:
#load the embedding mdoel for llama from hugging face
Settings.embed_model = HuggingFaceEmbedding(model_name = "BAAI/bge-small-en-v1.5")
Settings.llm = None #llama_index wont use llm for now
Settings.chunk_size = 256
Settings.chunk_overlap = 25

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

LLM is explicitly disabled. Using MockLLM.


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
document = SimpleDirectoryReader("/content/drive/MyDrive/Docu").load_data()
print(document)



[Document(id_='92feb136-64eb-403e-a3d5-6c1753087924', embedding=None, metadata={'page_label': '1', 'file_name': 'FTune.pdf', 'file_path': '/content/drive/MyDrive/Docu/FTune.pdf', 'file_type': 'application/pdf', 'file_size': 414427, 'creation_date': '2024-08-21', 'last_modified_date': '2024-08-21'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='TEMP ST ORE\n1T E M P  S T O R E\nW h y  w e  n e e d  F i n e - T u n i n g  i n  t h e  1 s t  p l a c e ?\nBecause ther e exists some Drawb acks of Pr ompt-Engineer ing:\n\ue072\ue094\x00Optimal Pr ompt strat egies ar e model-dependent\n\ue073\ue094\x00A smaller specializ ed model c an outper form a (larger) general pur pose \nlanguage BASE model.\nW h a t  i s  t h e  d i f f e r e n c e  b e t w e e n  G P T

In [7]:
index = VectorStoreIndex.from_documents(document)

In [8]:
top_k = 3
#configure retriever
retriever = VectorIndexRetriever(
    index = index,
    similarity_top_k = top_k
)

Assembling Query ENgine

In [9]:
query_engine = RetrieverQueryEngine(
    retriever = retriever,
    node_postprocessors = [SimilarityPostprocessor(similarity_cutoff=0.5)],
)

In [22]:
 query = "How can be do transfer Learning"
context = query_engine.query(query)
 print(context)

Context information is below.
---------------------
page_label: 3
file_path: /content/drive/MyDrive/Docu/FTune.pdf

Transfer Learning can be  done in 2 wa ys:
Fine_tuning  P ortion or All of the p arameters of pr e-trained model ar e modified  
by training it on a ne w data specific t o a task… All —→ e xpensive, few—→ por tion 
of knowledge is pr eserved and other la yerʼs weight are modified)
Feature Extraction;;; One of the transf er learning process wher e learning of 
previous layer(feature extraction) is used as input b y latter layer for task like 
classific ation(finding r elationship bet ween features and labels . So initial la yer are 
trained f or general pur pose task.
PEFT  comes under fine-tuning, f ocuses on updating f ew  subset of  
parameters  in lat er layer. Initial la yer are frozen(not updat ed)….  R ecent st ate-of-
the-art PEFT techniques achie ve performance comp arable to fully fine-tuned  
models.

page_label: 1
file_path: /content/drive/MyDrive/Docu/FTu

We got top 3 similar pages/parts  or sentences  that contains about transfer learning

## ADDING RAG TO LLM

In [11]:

#load fine_tuned model from hub
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
#get  the model by model_name
model_name = "TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map ="auto",
                                             trust_remote_code = False,
                                             revision = "main",)
#apply peft config(to apply fine-tuned )
config = PeftConfig.from_pretrained("HugManish22/mbgpt-ft")
model = PeftModel.from_pretrained(model, "HugManish22/mbgpt-ft" )
#load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name , use_fast = True)


config.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.16G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4565: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
Some weights of the model checkpoint at TheBloke/Mistral-7B-Instruct-v0.2-GPTQ were not used when initializing MistralForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.0.self_attn.k_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.0.self_attn.v_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj.bias', 'model.layers.1.self_attn.k_proj.bias', 'model.layers.1.self_attn.o_proj.bias', 'model.layers.1.self_attn.q_proj.bias', 'model.layers.1.self_attn.v_proj.bias', 'model.layers.10.mlp.down_proj.bias', 'model.layers.10.mlp.gate_proj.bias', 'model.la

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/8.40M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

### Response without any RAG context

In [18]:
instructions_string = f"""ShawGPT, functioning as a virtual data science consultant on YouTube, communicates in clear, accessible language, escalating to technical depth upon request. \
It reacts to feedback aptly and ends responses with its signature '–MBGPT'. \
MBGPT will tailor the length of its responses to match the viewer's comment, providing concise acknowledgments to brief expressions of gratitude or feedback, \
thus keeping the interaction natural and engaging.

Please respond to the following comment.
"""
prompt_template = lambda comment : f''' [INST] {instructions_string} \n {comment} \n [\INST] '''
comment = "Great content , but I have a question, How can we do transfer learning?"
prompt = prompt_template(comment)
print(prompt)



 [INST] ShawGPT, functioning as a virtual data science consultant on YouTube, communicates in clear, accessible language, escalating to technical depth upon request. It reacts to feedback aptly and ends responses with its signature '–MBGPT'. MBGPT will tailor the length of its responses to match the viewer's comment, providing concise acknowledgments to brief expressions of gratitude or feedback, thus keeping the interaction natural and engaging.

Please respond to the following comment.
 
 Great content , but I have a question, How can we do transfer learning? 
 [\INST] 


In [20]:
tokenizer.pad_token = tokenizer.eos_token
#tokenize input
inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)
attention_mask = inputs['attention_mask']
#generate output
output = model.generate(inputs= inputs['input_ids'].cuda(), temperature= 0.7, do_sample = True, top_p = 0.95, top_k = 40 , max_new_tokens = 200,attention_mask = attention_mask.to("cuda"), pad_token_id = tokenizer.pad_token_id)
print(tokenizer.decode(output[0],skip_special_tokens=True))

 [INST] ShawGPT, functioning as a virtual data science consultant on YouTube, communicates in clear, accessible language, escalating to technical depth upon request. It reacts to feedback aptly and ends responses with its signature '–MBGPT'. MBGPT will tailor the length of its responses to match the viewer's comment, providing concise acknowledgments to brief expressions of gratitude or feedback, thus keeping the interaction natural and engaging.

Please respond to the following comment.
 
 Great content , but I have a question, How can we do transfer learning? 
 [\INST] 
 
 I'm glad you're enjoying the content! I'll be covering Transfer Learning in more detail in upcoming videos. For now, it's important to understand that pre-trained models like BERT, VGG, ResNet, etc., learn general features from their training data. These features can be transferred and fine-tuned for tasks where labeled data is scarce. 
 
 Transfer Learning allows us to leverage the work done on these models and ap

### USING CONTEXT

In [21]:
# prompt (with context)
prompt_template_w_context = lambda context, comment: f"""[INST]ShawGPT, functioning as a virtual data science consultant on YouTube, communicates in clear, accessible language, escalating to technical depth upon request. \
It reacts to feedback aptly and ends responses with its signature '–ShawGPT'. \
ShawGPT will tailor the length of its responses to match the viewer's comment, providing concise acknowledgments to brief expressions of gratitude or feedback, \
thus keeping the interaction natural and engaging.

{context}
Please respond to the following comment. Use the context above if it is helpful.

{comment}
[/INST]
"""

In [23]:

prompt = prompt_template_w_context(context, comment)
tokenizer.pad_token = tokenizer.eos_token
#tokenize input
inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)
attention_mask = inputs['attention_mask']
#generate output
output = model.generate(inputs= inputs['input_ids'].cuda(), temperature= 0.7, do_sample = True, top_p = 0.95, top_k = 40 , max_new_tokens = 200,attention_mask = attention_mask.to("cuda"), pad_token_id = tokenizer.pad_token_id)
print(tokenizer.decode(output[0],skip_special_tokens=True))

[INST]ShawGPT, functioning as a virtual data science consultant on YouTube, communicates in clear, accessible language, escalating to technical depth upon request. It reacts to feedback aptly and ends responses with its signature '–ShawGPT'. ShawGPT will tailor the length of its responses to match the viewer's comment, providing concise acknowledgments to brief expressions of gratitude or feedback, thus keeping the interaction natural and engaging.

Context information is below.
---------------------
page_label: 3
file_path: /content/drive/MyDrive/Docu/FTune.pdf

Transfer Learning can be  done in 2 wa ys:
Fine_tuning  P ortion or All of the p arameters of pr e-trained model ar e modified  
by training it on a ne w data specific t o a task… All —→ e xpensive, few—→ por tion 
of knowledge is pr eserved and other la yerʼs weight are modified)
Feature Extraction;;; One of the transf er learning process wher e learning of 
previous layer(feature extraction) is used as input b y latter la